In [1]:
import torch
import numpy as np
from pathlib import Path
#Create memory mapped class using PtychoDataset
from ptycho_torch.dset_loader_pt_mmap import PtychoDataset

/root/miniconda3/envs/ptychopinn_ptychodus_pytorch/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


The purpose of this notebook is to test different pytorch functions/functionalities to account for when implementing Pytchopinn helper functions

# Translations

Following documentation from: [stackoverflow](https://stackoverflow.com/questions/72769563/shifting-an-image-with-bilinear-interpolation-in-pytorch)

In [12]:
test_im = torch.rand(5,2,10,10) #(N, C, H, W)
n, c, h, w = test_im.shape

#Create x, y stacked meshgrid
x, y = torch.arange(h)/(h-1), torch.arange(w)/(w-1)

#Grid
grid = torch.stack(torch.meshgrid(x, y, indexing = 'ij')) * 2 - 1
#Adding additional dimensions so grid sampling
grid = grid[None].repeat(5,1,1,1)




True

In [6]:
#Testing numpy indexing
#Create 3 dimensional random numpy array
test_np = np.random.rand(10)
test_indices = np.random.randint(0,9,size=(10,4))
test_np, test_np[test_indices]

(array([0.47128105, 0.84978654, 0.44762854, 0.98600908, 0.69289631,
        0.30358325, 0.95456834, 0.39048805, 0.22445553, 0.05974865]),
 array([[0.47128105, 0.95456834, 0.69289631, 0.47128105],
        [0.30358325, 0.30358325, 0.30358325, 0.39048805],
        [0.84978654, 0.47128105, 0.69289631, 0.39048805],
        [0.98600908, 0.30358325, 0.47128105, 0.69289631],
        [0.69289631, 0.69289631, 0.47128105, 0.84978654],
        [0.39048805, 0.22445553, 0.39048805, 0.98600908],
        [0.39048805, 0.39048805, 0.30358325, 0.30358325],
        [0.30358325, 0.44762854, 0.44762854, 0.22445553],
        [0.22445553, 0.22445553, 0.22445553, 0.98600908],
        [0.84978654, 0.39048805, 0.30358325, 0.69289631]]))

In [13]:
from ptycho_torch.patch_generator import get_neighbor_indices, sample_rows

In [9]:
test_x = np.random.rand(100)
test_y = np.random.rand(100)

In [10]:
nn_indices = get_neighbor_indices(test_x, test_y, 6)

In [17]:
sampled_rows = sample_rows(nn_indices, 4, 10).reshape(-1,4)
sampled_rows.shape

(1000, 4)

In [33]:
test_x[sampled_rows].shape

(1000, 4)

In [34]:
coords_nn = np.stack([test_x[sampled_rows],
                            test_y[sampled_rows]],axis=2)[:, :, None, :]

coords_nn.shape

(1000, 4, 1, 2)

In [22]:
dummy_images = np.random.rand(100,10,10)
sample_dummy_images = dummy_images[nn_indices]

In [24]:
dummy_images.shape, nn_indices.shape, sample_dummy_images.shape

((100, 10, 10), (100, 7), (100, 7, 10, 10))

In [37]:
coords_nn_avg = np.mean(coords_nn, axis=1)
coords_nn[0,...],coords_nn_avg[0,...]

(array([[[0.48472418, 0.83961076]],
 
        [[0.43905179, 0.62763275]],
 
        [[0.49147743, 0.834936  ]],
 
        [[0.35655118, 0.901465  ]]]),
 array([[0.44295114, 0.80091113]]))

In [4]:
import os

In [5]:
os.getcwd()

'/home/av_linux/PtychoPINN/tests'

In [48]:
#Numpy load file /notebooks/train_data.npz

#Load file
train_data = np.load('../notebooks/train_data.npz')
test_data = np.load('../notebooks/test_data.npz')

In [5]:
train_data.files

['xcoords',
 'ycoords',
 'xcoords_start',
 'ycoords_start',
 'diff3d',
 'probeGuess',
 'objectGuess',
 'scan_index']

In [56]:
train_data['xcoords_start'][0]

37.61779245164779

In [9]:
#Testing nn_indice indexing
from ptycho_torch.patch_generator import get_neighbor_indices, sample_rows, group_coords


In [10]:
#Create fake 20 x 2 array of random numbers between 0 and 1
N = 1000
test_coords = np.random.rand(N,2)
#Create fake images in 20 x 40 x 40 array
test_images = np.random.rand(N,40,40)

#Create fake nn_indices
nn_indices = get_neighbor_indices(test_coords[:,0], test_coords[:,1], 6)

In [11]:
#Sample rows 
sampled_rows = sample_rows(nn_indices, 4, 10).reshape(-1,4)

sampled_rows.shape

(10000, 4)

In [4]:
x_coords = test_coords[:,0]
y_coords = test_coords[:,1]

nn_indices, coords_nn = group_coords(nn_indices, x_coords, y_coords, 4)

In [43]:
coords_nn.shape

(5, 4, 1, 2)

In [29]:
#Testing out memory mapping to random tensor
from tensordict import MemoryMappedTensor, TensorDict

In [33]:
#Small tensordict
test_tensor = TensorDict({'image': MemoryMappedTensor.empty(
                            (20,4,40,40), dtype=torch.float64),
                          'coords': MemoryMappedTensor.empty(
                            (20,4,1,2), dtype=torch.float64)},
                            batch_size = 20)

test_tensor.memmap_(prefix="../data/test_memmap")

#Populate it
for i in range(20):
    test_tensor['image'][i] = torch.from_numpy(test_images[nn_indices][i])
    test_tensor['coords'][i] = torch.from_numpy(coords_nn[i])


In [61]:
train_data['diff3d'][0].dtype

dtype('float64')

# Creating dummy data to test out ptychodataset

In [12]:
#Creating dummy npz datasets for testing out PtychoDataset in data/dummy_data
n_dummies = 10
for i in range(n_dummies):
    #Set # of images from 5000 to 25000 (integers only)
    num_images = np.random.randint(100,300)
    #Create randomized image stack of dim (num_images, 128, 128)
    dummy_images = np.random.rand(num_images,128,128)
    #Create randomized coordinates (x,y) between 0 and 60 (arbitrary)
    xcoords = np.random.rand(num_images) * 59 + 1
    ycoords = np.random.rand(num_images) * 59 + 1

    #Create fake xcoords_start and y_coords start. They should have slightly smaller values than
    #coords_x and coords_y, but not go below 0
    xcoords_start = xcoords - np.random.rand(num_images)
    xcoords_start[xcoords_start < 0] = 0

    ycoords_start = ycoords - np.random.rand(num_images)
    ycoords_start[ycoords_start < 0] = 0

    #Create random probe function
    probe = np.random.rand(128,128)
    
    #Save following arrays into npz file named based on dummy number in '../data/dummy_data'
    np.savez('../data/dummy_data_small/dummy_data_{}'.format(i), 
             diff3d = dummy_images,
             xcoords = xcoords,
             ycoords = ycoords,
             xcoords_start = xcoords_start,
             ycoords_start = ycoords_start,
             probe = probe)


# Testing out PtychoDataset

In [2]:
dset_params = {'grid_size': (2,2),
               'probe_map': [0,1,0,2,1,2,1,0,2,2],
               'probes': np.random.rand(3,128,128),
               'K': 6,
               'n_subsample': 10,
               'C': None}

#Create ptychodataset object
test_dset = PtychoDataset('../data/dummy_data_small', '', dset_params)

Creating memory mapped tensor dictionary...
Memory map length: 21540
Memory map creation time: 10.209872245788574
Populating memory map for dataset 0
Non-diffraction memory map write time: 0.00074005126953125
global_from: 0, global_to: 1024
j: 0, local_to: 1024
global_from: 1024, global_to: 2048
j: 1024, local_to: 2048
global_from: 2048, global_to: 2330
j: 2048, local_to: 2330
Diffraction memory map write time: 2.4173295497894287
Populating memory map for dataset 1
Non-diffraction memory map write time: 0.0003204345703125
global_from: 2330, global_to: 3354
j: 0, local_to: 1024
global_from: 3354, global_to: 4378
j: 1024, local_to: 2048
global_from: 4378, global_to: 5180
j: 2048, local_to: 2850
Diffraction memory map write time: 2.396333694458008
Populating memory map for dataset 2
Non-diffraction memory map write time: 0.011011838912963867
global_from: 5180, global_to: 6204
j: 0, local_to: 1024
global_from: 6204, global_to: 7228
j: 1024, local_to: 2048
global_from: 7228, global_to: 7650

In [10]:
import time

In [11]:
#Create array of 64 random numbers from 0 to 21540
test_indices = np.random.randint(0,21540,size=64)

#Time dataloading
start = time.time()
a = test_dset.mmap_ptycho[test_indices]
end = time.time()
print(end-start)

0.08963155746459961
